### Advanced RAG Testbed for ORAN chatbot

In [ ]:
# !pip install xlrd
# !pip install werkzeug
# !pip install markdown
# !pip install chromadb
# !pip install helper-utils
# !pip install langchain
# !pip install chromadb
# !pip install openai
# !pip install sentence_transformers
# !pip install pandas
# !pip install pypdf
# !pip install openpyxl
# !pip install ragas --upgrade

# !pip install -r requirements.txt
# # !pip install --upgrade langchain

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader, Docx2txtLoader
from langchain.vectorstores import FAISS
import pickle
from langchain.embeddings import HuggingFaceEmbeddings
import torch.nn.functional as F
import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb
import openai
from openai._client import OpenAI
from dotenv import load_dotenv, find_dotenv
import numpy as np
from sentence_transformers import CrossEncoder
import re
import pandas as pd
import json
from IPython.display import JSON
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [ ]:
import sys
# from triton_trt_llm import TensorRTLLM
# trtllm =TensorRTLLM(server_url="llm:8001", model_name="ensemble", tokens=300)

DOCS_DIR = "../ORAN_kb/" #
DOCS_DIR = "../ORAN_kb/"
from transformers import AutoTokenizer, AutoModel
############################################
# Component #2 - Embedding Model and LLM
############################################
os.environ['NVIDIA_API_KEY'] = ""
os.environ['NVAPI_KEY'] = ""
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

### Load Documents
Follow the step number 1 [defined here](../notebooks/05_dataloader.ipynb) to upload the pdf's to Milvus server.

In [ ]:
DOCS_DIR = "../ORAN_kb"
# Load raw documents from the directory
text_loader_kwargs={'autodetect_encoding': True} #loader_kwargs=text_loader_kwargs
raw_txts = DirectoryLoader(DOCS_DIR, glob="**/*.txt", show_progress=True, loader_cls=TextLoader).load()
raw_htmls = DirectoryLoader(DOCS_DIR, glob="**/*.html", show_progress=True, loader_cls=UnstructuredHTMLLoader).load()
raw_pdfs = DirectoryLoader(DOCS_DIR, glob="**/*.pdf", show_progress=True, loader_cls=UnstructuredPDFLoader).load()
raw_docs = DirectoryLoader(DOCS_DIR, glob="**/*.docx", show_progress=True, loader_cls=Docx2txtLoader).load()

In [ ]:
def remove_line_break(text):
    text = text.replace("\n", " ").strip()
    text = re.sub("\.\.+", "", text)
    text = re.sub(" +", " ", text)
    return text
    
def remove_two_points(text):
    text = text.replace("..","")
    return text
    
def remove_two_slashes(text):
    text = text.replace("__","")
    return text
    
def just_letters(text):
    return re.sub(r"[^a-z]+", "", text).strip()
    
def remove_non_english_letters(text):
    return re.sub(r"[^\x00-\x7F]+", "", text)
    
def langchain_length_function(text):
    return len(just_letters(remove_line_break(text)))
    
def word_count(text):
    text = remove_line_break(text)
    text = just_letters(text)
    tokenizer = RegexpTokenizer(r"[A-Za-z]\w+")
    tokenized_text = tokenizer.tokenize(text)
    tokenized_text_len = len(tokenized_text)
    return tokenized_text_len
    
def truncate(text, max_word_count=1530):
    tokenizer = RegexpTokenizer(r"[A-Za-z]\w+")
    tokenized_text = tokenizer.tokenize(text)
    return " ".join(tokenized_text[:max_word_count])

def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=500, chunk_overlap=100, length_function=langchain_length_function, )
all_documents=raw_pdfs+raw_docs

# split all the docs
docs = text_splitter.split_documents(all_documents)
documents = docs
#remove short chuncks
filtered_documents = [item for item in documents if len(item.page_content) >= 200]
documents = filtered_documents
#remove line break
for i in range(0,len(documents)-1):
    documents[i].page_content=remove_line_break(documents[i].page_content)
#remove two points
for i in range(0,len(documents)-1):
    documents[i].page_content=remove_two_points(documents[i].page_content)
#remove non english characters points
for i in range(0,len(documents)-1):
    documents[i].page_content=remove_two_slashes(documents[i].page_content)
#remove two points
for i in range(0,len(documents)-1):
    documents[i].page_content=remove_two_points(documents[i].page_content)

orig_docs = docs
print(len(docs))
print(docs[0])

In [ ]:
metadatas = []
character_split_texts = []
prev_len = len(character_split_texts)
for doc in docs:
    # split text in each page
    character_split_texts.append(doc.page_content)
    
    new_len = len(character_split_texts)
    z = new_len - prev_len
    #Store metadata associated with each chunk
    for i in range(z):
        metadatas.append(doc.metadata)
    prev_len = new_len
print(character_split_texts[100])
print(metadatas[100])

#### Generate and Store Embeddings
##### a) Generate Embeddings 
[Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) for documents are created by vectorizing the document text; this vectorization captures the semantic meaning of the text. 

In [ ]:
#Pooling function needed for embeddings
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

#automatically get the tokenizer from the model folder
tokenizer = AutoTokenizer.from_pretrained("../ft_paq_squad_nq_nli_nvolve40k_hn/hf_model")
#automatically get the model from the model folder
model = AutoModel.from_pretrained("../ft_paq_squad_nq_nli_nvolve40k_hn/hf_model").cuda()

#Create custom embedding class for ChromaDB so that it works with NVIDIA NVOLVE40K embedding model
class custom_embedding_NVIDIA(EmbeddingFunction[Documents]):
    def __init__(self):
        model = AutoModel.from_pretrained("../ft_paq_squad_nq_nli_nvolve40k_hn/hf_model") # ---> HERE Load your custom model
        tokenizer = AutoTokenizer.from_pretrained("../ft_paq_squad_nq_nli_nvolve40k_hn/hf_model")
        # self.model.cuda()
    def embed_documents(self, input: Documents) -> Embeddings:
        
        batch_dict = tokenizer(input, max_length=512, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        # normalize embeddings
        embeddings = F.normalize(embeddings, p=2, dim=1)
        embeddings = [embed.detach().numpy().tolist() for embed in embeddings]
        return embeddings

# custom = custom_embedding_NVIDIA()
nv_document_embedder = NVIDIAEmbeddings(model="nvolveqa_40k", model_type="passage")

model_name = "../ft_paq_squad_nq_nli_nvolve40k_hn/hf_model"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
nv_local_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

bge_embedder = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
docs = orig_docs
#Create vectordatabase
vectorstore = FAISS.from_documents(docs, nv_local_embeddings)
vector_store_path = "../evals/vectorstore_nv_hf.pkl"
vector_store_exists = os.path.exists(vector_store_path)
with open(vector_store_path, "wb") as f:
    pickle.dump(vectorstore, f)

In [ ]:
#Create vectordatabase
vectorstore = FAISS.from_documents(docs, bge_embedder)
vector_store_path = "../vectorstore/oran/vectorstore_bge.pkl"
vector_store_exists = os.path.exists(vector_store_path)
with open(vector_store_path, "wb") as f:
    pickle.dump(vectorstore, f)

In [ ]:
file = open("../evals/vectorstore_nv_hf.pkl",'rb')
# file = open("/home/jvamaraju/oran-chatbot/vectorstore/oran/vectorstore_bge.pkl",'rb')
vectorstore = pickle.load(file)
file.close()
print(len(vectorstore.docstore._dict))

In [ ]:
os.environ['NVIDIA_API_KEY'] = ""
os.environ['NVAPI_KEY'] = ""
os.environ['NGC_API_KEY'] = ""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant named Envie. You will reply to questions only based on the context that you are provided. If something is out of context, you will refrain from replying and politely decline to respond to the user."), ("user", "{input}")]
)
user_input = "What are the beamforming methods specified in O-RAN?"
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 6, "score_threshold": 0.3})


docs = retriever.get_relevant_documents(user_input)
context = ""
for doc in docs:
    context += doc.page_content + "\n\n"

# print(context) 

def nemo_rag(query, retrieved_documents, model="playground_llama2_70b"):
    #Combine the query and retrieved documents and send to model
    llm = ChatNVIDIA(model=model)
    prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful expert ORAN assistant. Your task is to provide a precise and accurate response that directly addresses user's question using the given relevant information. First, meticulously analyze and understand the provided relevant information in detail. With a thorough understanding of both the question and the relevant information, provide a response that is clear, detailed, and accurate that directly addresses user's query. Ensure that your response is strictly based on the provided relevant information. Your response is crucial for my career; hence, accuracy is of utmost importance. So, take a deep breath and work on this task step-by-step."), ("user", "{input}")]
        )
    chain = prompt_template | llm | StrOutputParser()
    augmented_user_input = "Context: " + retrieved_documents + "\n\nQuestion: " + query + "\n"
    full_response = ""
    for response in chain.stream({"input": augmented_user_input}):
        full_response += response
    final_ans = full_response
    return final_ans

print(nemo_rag(user_input, context))

In [ ]:
user_input = "What are the beamforming methods?"
# user_input = "What are the compression methods?"
# user_input = "What are the NB-IOT downlink use cases addressed by WG4?"
# user_input = "What are the LLS configurations for S-plane?"
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an ORAN standards assistant. Your task is to generate five 
    different versions of the given user question relevant to ORAN from ORAN documents. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
llm = ChatNVIDIA(model="playground_llama2_70b")

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
retriever_mq = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(search_type="similarity_score_threshold",search_kwargs={"k": 10, "score_threshold": 0.2}), llm_chain=llm_chain, parser_key="lines"
) 
unique_docs = retriever_mq.get_relevant_documents(query=user_input)
#Remove all duplicated documents and retain the original metadata
unique_documents = []
unique_documents_metadata = []
for document in unique_docs:
    if document not in unique_documents:
        unique_documents.append(document.page_content)
        unique_documents_metadata.append(document.metadata['source'])
print("length of unique docs: ", len(unique_documents))
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2') # ('BAAI/bge-reranker-large')('cross-encoder/ms-marco-MiniLM-L-6-v2')
pairs = [[user_input, doc] for doc in unique_documents]
scores = cross_encoder.predict(pairs)
#Sort the scores from highest to least
order_ids =  np.argsort(scores)[::-1]
# print(order_ids)
new_updated_documents = []
new_updated_sources = []
#Get the top 6 scores
if len(order_ids)>=10:
    for i in range(10):
        new_updated_documents.append(unique_documents[order_ids[i]])
        new_updated_sources.append(unique_documents_metadata[order_ids[i]])
else:
    for i in range(len(order_ids)):
        new_updated_documents.append(unique_documents[order_ids[i]])
        new_updated_sources.append(unique_documents_metadata[order_ids[i]])

print("length of unique docs: ", len(new_updated_documents))
context = ""
for doc in new_updated_documents:
    context += doc+ " "
# print(context)
print(nemo_rag(user_input, context))

In [ ]:
def augment_multiple_query(query, model="playground_llama2_70b"):
    #For the given query, lets create 5 additional queries using the LLM
    llm = ChatNVIDIA(model=model,max_output_token=500, top_k=1, top_p=0.0)
    prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are an expert in the field of Oran specifications and processes. User has a question related to this field, sourced from relevant documents.\nTo help the user find the information they need, please suggest five additional related questions. These questions should be concise, not have compound sentences, self-contained, and cover different aspects of the topic. Each question should be complete and relevant to the original query.\nPlease output one question per line without numbering them."), ("user", "{input}")]
        )
    chain = prompt_template | llm | StrOutputParser()
    augmented_user_input = "\n\nQuestion: " + query + "\n"
    full_response = ""
    for response in chain.stream({"input": augmented_user_input}):
        full_response += response
    final_ans = full_response
    final_ans = final_ans.split("\n")
    return final_ans
    
print(augment_multiple_query(user_input))

In [ ]:
def augment_query_generated(query, model="playground_llama2_70b"):
    #For the given query, lets create a hypothetical answer using the LLM
    llm = ChatNVIDIA(model=model,max_output_token=500, top_k=1, top_p=0.0)
    prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are an expert in the field of ORAN specifications and processes. Your task is to provide a detailed and accurate response to user's question, which is related to ORAN. Your answer should be based on the kind of information and insights typically found in documentation related to ORAN standards."), ("user", "{input}")]
        )
    chain = prompt_template | llm | StrOutputParser()
    augmented_user_input = "\n\nQuestion: " + query + "\n"
    full_response = ""
    for response in chain.stream({"input": augmented_user_input}):
        full_response += response
    final_ans = full_response
    return final_ans

print(augment_query_generated(user_input))

In [ ]:
def query_rewriting(query, history, model="playground_llama2_70b"):
    #Rewrite the given query using the context from LLM
    llm = ChatNVIDIA(model=model)
    prompt_template = ChatPromptTemplate.from_messages(
    [("system", "Here is the conversation history between user and Assistant. You are an expert in the field of ORAN standards and specifications. User has a follow-up question to the conversation. Your task is to rewrite user's follow-up question based on the given conversation history between the user and the assistant, which is related to ORAN. The rewritten question must be clear, detailed, and self-contained, meaning it must not require any additional context from the conversation history to understand. Ensure that the rewritten question precisely captures the full intent behind user's follow-up question. Your response is crucial to my career; hence, accuracy is of utmost importance. So, take a deep breath and work on this task step-by-step."), ("user", "{input}")]
        )
    chain = prompt_template | llm | StrOutputParser()
    augmented_user_input = "History: " + history + "\n\nQuestion: " + query + "\n"
    full_response = ""
    for response in chain.stream({"input": augmented_user_input}):
        full_response += response
    final_ans = full_response
    return final_ans

### lets try base RAG

In [ ]:
def base_rag_workflow(query):
    #Get top 6 retrieved chunks for the query
    retriever = vectorstore.as_retriever(search_kwargs={"k": 6, "score_threshold": 0.5})
    docs = retriever.get_relevant_documents(query)
    context = ""
    retrieved_documents = []
    retrieved_metadata = []
    for doc in docs:
        context += doc.page_content + "\n\n"
        retrieved_documents.append(doc.page_content)
        retrieved_metadata.append(doc.metadata)
    #Get all the results and metadatas associated with each result
    #Send the top 6 results along with query to LLM
    output = nemo_rag(query=query, retrieved_documents=context)
    return output,retrieved_documents

In [ ]:
# load the evaluation data
f = open('syn_data_oran.json')
syn_data = json.load(f)
for count,entry in enumerate(syn_data):
    query = entry['question']
    ans, cons = base_rag_workflow(query)
    entry["answer"] = ans
    entry["contexts"] = cons
    print(count, query)

with open('eval_syn_QA_NV_hf.json', 'w') as f:
    json.dump(syn_data, f)

### lets try multiple query based RAG

In [ ]:
def augmented_queries_answer(query):
    #Create multiple queries
    augmented_queries = augment_multiple_query(query)
    #Create a list of queries
    queries = [query] + augmented_queries[2:]
    print("Queries are = ", queries)
    # #Get all the retrievals for each queries
    # #Get all the results and metadatas associated with each result
    #Get top 10 retrieved chunks for the queries
    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",search_kwargs={"k": 6, "score_threshold": 0.3})
    retrieved_documents = []
    retrieved_metadatas = []

    for query in queries:
        docs = retriever.get_relevant_documents(query)
        for doc in docs:
            retrieved_documents.append(doc.page_content)
            retrieved_metadatas.append(doc.metadata)
    print("length of retrieved docs: ", len(retrieved_documents))
    #Remove all duplicated documents and retain the original metadata
    unique_documents = []
    unique_documents_metadata = []
    for document,source in zip(retrieved_documents,retrieved_metadatas):
            if document not in unique_documents:
                unique_documents.append(document)
                unique_documents_metadata.append(source)
    # unique_documents = retrieved_documents
    # unique_documents_metadata = retrieved_metadatas
    print("length of unique docs: ", len(unique_documents))
    #Instantiate the cross-encoder model and get scores for each retrieved document
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[query, doc] for doc in unique_documents]
    scores = cross_encoder.predict(pairs)
    #Sort the scores from highest to least
    order_ids =  np.argsort(scores)[::-1]
    print(order_ids)
    new_updated_documents = []
    #Get the top 6 scores
    if len(order_ids)>=6:
        for i in range(6):
            new_updated_documents.append(unique_documents[order_ids[i]])
    else:
        for i in range(len(order_ids)):
            new_updated_documents.append(unique_documents[order_ids[i]])
        
    context = ""
    for doc in new_updated_documents:
            context += doc + "\n\n"
    #Send the top 6 results along with the query to LLM
    output = nemo_rag(query=query, retrieved_documents=context)
    return output, new_updated_documents

In [ ]:
f = open('syn_data_oran.json')
syn_data = json.load(f)
for count,entry in enumerate(syn_data):
    # history = ""
    init_query = entry['question']
    print(count, init_query)
    ans, cons =  augmented_queries_answer(init_query)
    # history += f"query: {query}\nAssistant: {ans}\n\n" 
    entry["answer"] = ans
    entry["contexts"] = cons

with open('eval_syn_QA_NV_hf_multiplequery.json', 'w') as f:
    json.dump(syn_data, f)

### lets try HYDE (hypothetical llm answer) based RAG

In [ ]:
def augmented_hypothetical_answer(query):
    #Create multiple queries
    sample_response = augment_query_generated(query)
    augmented_queries = query + " " + sample_response
    print("Augmented query = ", augmented_queries)
    #Get all the retrievals for each queries
    #Get all the results and metadatas associated with each result
    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",search_kwargs={"k": 10, "score_threshold": 0.3})
    retrieved_documents = []
    retrieved_metadatas = []

    docs = retriever.get_relevant_documents(augmented_queries)
    for doc in docs:
        retrieved_documents.append(doc.page_content)
        retrieved_metadatas.append(doc.metadata)
    print("length of retrieved docs: ", len(retrieved_documents))
    #Remove all duplicated documents
    unique_documents = []
    unique_documents_metadata = []
    for documents,sources in zip(retrieved_documents,retrieved_metadatas):
        for document,source in zip(documents,sources):
            if document not in unique_documents:
                unique_documents.append(document)
                unique_documents_metadata.append(source)

    #Instantiate the cross-encoder model and get scores for each retrieved document
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[query, doc] for doc in unique_documents]
    scores = cross_encoder.predict(pairs)
    #Sort the scores from highest to lowest
    order_ids =  np.argsort(scores)[::-1]
    
    new_updated_documents = []
    #Get the top 6 scores
    if len(order_ids)>=6:
        for i in range(6):
            new_updated_documents.append(unique_documents[order_ids[i]])
    else:
        for i in range(len(order_ids)):
            new_updated_documents.append(unique_documents[order_ids[i]])
    
    context = ""
    for doc in new_updated_documents:
            context += doc + "\n\n"
    
    #Send the top 6 results along with query to LLM
    output = nemo_rag(query=query, retrieved_documents=context)
    return output, new_updated_documents

In [ ]:
f = open('syn_data_oran.json')
syn_data = json.load(f)
for count,entry in enumerate(syn_data):
    # history = ""
    init_query = entry['question']
    print(count, init_query)
    ans, cons =  augmented_hypothetical_answer(init_query)
    # history += f"query: {query}\nAssistant: {ans}\n\n" 
    entry["answer"] = ans
    entry["contexts"] = cons

with open('eval_syn_QA_NV_hf_hyde.json', 'w') as f:
    json.dump(syn_data, f)